In [1]:
import json
import pymongo
from bson import ObjectId

client = pymongo.MongoClient('localhost', 27017)
# create db tutorial
mydb = client["homework"]
# create collection example
collection = mydb["pubmed"]
with open('Chap3/pubmed_cleaned.json') as file:
    file_data = json.load(file)

In [6]:
for document in file_data:
    if '_id' in document and '$oid'in document['_id']:
        document['_id']= ObjectId(document['_id']['$oid'])
collection.insert_many(file_data)    

In [16]:
#1)Create an index, explain your choice of key.
collection.create_index([ ("pmid",1) ])

'pmid_1'

In [22]:
#2)Delete every paper that was published prior 2019
result = collection.delete_many({"date": {"$lt": "year 2019, month 1, day 1, hour 0, minute 0"}})
print(f"Nombre de documents supprimés: {result.deleted_count}")

Nombre de documents supprimés: 5


In [2]:
num_single_author_papers = collection.count_documents({"authors": {"$regex": "name ml"}})
print(f"Nombre de documents avec un seul auteur : {num_single_author_papers}")

Nombre de documents avec un seul auteur : 99409


In [14]:
# 3)How many paper have a single author ? Two authors ?
num_single_author_documents = 0
num_two_authors_documents = 0

# Compter le nombre de documents avec un seul auteur et deux auteurs
for document in collection.find():
    authors = document['authors'].split('\n')
    num_authors = len(authors)
    if num_authors == 1:
        num_single_author_documents += 1
    elif num_authors == 2:
        num_two_authors_documents += 1

# Afficher les résultats
print("Nombre de documents avec un seul auteur :", num_single_author_documents)
print("Nombre de documents avec deux auteurs :", num_two_authors_documents)

Nombre de documents avec un seul auteur : 5480
Nombre de documents avec deux auteurs : 10153


In [21]:
#What's the last paper inserted in the db ?
x = collection.find_one(sort=[('_id', -1)])
print("Last Paper:")
print("Title:", last_paper['title'])
print("Authors:", last_paper['authors'])
print("Date:", last_paper['date'])

{'_id': ObjectId('60c753bac4b7559e040fe2b3'), 'pmid': 31226374, 'title': 'Identification of a novel UDP-glycosyltransferase gene from Rhodiola rosea and its expression during biotransformation of upstream precursors in callus culture.', 'ISSN': '1879-0003', 'abstract': ' "Roseroot (Rhodiola rosea L.) is a medicinal plant with adaptogenic properties and several pharmaceutically important metabolites. In this study, a full length cDNA encoding a UDPG gene of roseroot was identified, cloned and characterized. Its ORF (1425bp) was transferred into E. coli, where the expression of the recombinant enzyme was confirmed. To monitor the enzyme activity, 3 precursors (tyramine, 4-hydroxyphenylpyruvate & tyrosol) of salidroside biosynthesis pathway were added to roseroot callus cultures and samples were harvested after 1, 6, 12, 24, 48 & 96h. Along with the controls (without precursor feeding), each sample was subjected to HPLC and qRT-PCR for phytochemical and relative UDP-glycosyltransferase ge

In [1]:
#5)Find articles with null meshwords.
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client['homework']
collection = db['pubmed']

# Find articles with null meshwords
articles_with_null_meshwords = collection.find({"meshwords": {"$in": [None, ""]}})

# Print the details of articles with null meshwords
print("Articles with null meshwords:")
for article in articles_with_null_meshwords:
    print("Title:", article['title'])

Articles with null meshwords:
Title: Liberating Native Mass Spectrometry from Dependence on Volatile Salt Buffers by Use of Gabor Transform.
Title: Palladium-Catalyzed Carbo-Oxygenation of Propargylic Amines using in Situ Tether Formation.
Title: Factors associated with distant metastasis in pediatric thyroid cancer: evaluation of the SEER database.
Title: Special Issue: Gut Bacteria-Mucus Interaction.
Title: Introduction to 2019 World Federation for Medical Education World Conference.
Title: Editorial for the Special Issue on Glassy Materials Based Microdevices.
Title: Comparative Behavioral Correlation of High and Low-Performing Mice in the Forced Swim Test.
Title: Correction: Comparison of school day eating behaviours of 8-11 year old children from Adelaide, South Australia, and London, England: Child eating behaviours in South Australia and England.
Title: Correction to: RNA G-quadruplexes at upstream open reading frames cause DHX36- and DHX9-dependent translation of human mRNAs.
T

Title: Annals of Palliative Medicine is now indexed in Science Citation Index Expanded (SCIE).
Title: Probing the Proton-Coupled Electron-Transfer (PCET) Reactivity of a Cross-Conjugated Cruciform Chromophore by Redox-State-Dependent Fluorescence.
Title: Exploration of Nano-Saturns: A Spectacular Sphere-Ring Supramolecular System.
Title: 1,2-Dihydro-1-hydroxy-2,3,1-benzodiazaborine Bearing an Acridine Moiety as a Circular Dichroism Probe for Determination of Absolute Configuration of Mono-Alcohols.
Title: Covalently Grafting Cobalt Porphyrin onto Carbon Nanotubes for Efficient CO2 Electroreduction.
Title: The Onset of Dehydrogenation in Solid Ammonia Borane: An Ab Initio Metadynamics Study.
Title: Corrigendum: An international systems-theoretic comparison of hospital accreditation: developing an implementation typology.
Title: Possible Routes for Efficient Thermo-Electric Energy Conversion in a Molecular Junction.
Title: Dual Resonator MEMS Magnetic Field Gradiometer.
Title: High-Sensi

Title: Dedifferentiated retroperitoneal large liposarcoma and laparoscopic treatment: Is it possible and safe? The first literature case report.
Title: Asymmetric transmission of elastic shear vertical waves in solids.
Title: Highly efficient ethylene glycol electrocatalytic oxidation based on bimetallic PtNi on 2D molybdenum disulfide/reduced graphene oxide nanosheets.
Title: Effects of methylation in acceptors on the hydrogen bond complexes between 2,2,2-trifluoroethanol and cyclic ethers.
Title: Facile Preparation of Snowflake-Like MnO2 @NiCo2 O4 Composites for Highly Efficient Electromagnetic Wave Absorption.
Title: Enantioselective Desymmetrization of 1,4-Dihydropyridines by Oxidative NHC Catalysis.
Title: Tertiary alpha-Silyl Alcohols by Diastereoselective Coupling of 1,3-Dienes and Acylsilanes Initiated by Enantioselective Copper-Catalyzed Borylation.
Title: Appraisal of re-irradiation for the recurrent glioblastoma in the era of MGMT promotor methylation.
Title: National trends

Title: Phosphonium-Ring-Fused Bicyclic Metallafuran Complexes of Ruthenium and Osmium.
Title: Regiochemistry Control by Bipyridine Substituents in the Deprotonation of Re(I) and Mo(II) N-Alkylimidazole Complexes.
Title: Highly Reproducible Formation of a Polymer Single-Molecule Junction for a Well-Defined Current Signal.
Title: Intermolecular C-H Amidation of (Hetero)arenes to Produce Amides through Rhodium-Catalyzed Carbonylation of Nitrene Intermediates.
Title: Phosphazenyl Phosphines: The Most Electron-Rich Uncharged Phosphorus Bronsted and Lewis Bases.
Title: Composition control in biphasic silicate microspheres on stimulating new bone regeneration and repair of osteoporotic femoral bone defect.
Title: Are Reactive Oxygen Species Important Mediators of Vascular Dysfunction?
Title: A Modular Access to Divinyldiphosphenes with a Strikingly Small HOMO-LUMO Energy Gap.
Title: Brodie\'s or Hummers\' Method: Oxidation Conditions Determine the Structure of Graphene Oxide.
Title: Manipulat

Title: Insights into the Kinetics of Supramolecular Comonomer Incorporation in Water.
Title: De Novo Duplication in the CHD7 Gene Associated With Severe CHARGE Syndrome.
Title: Opioids, Pain Management, and Palliative Care in a Norwegian Nursing Home From 2013 to 2018.
Title: New species of leaf-mining Phyllonorycter (LepidopteraGracillariidae) from Siberia feeding on Caragana (Fabaceae).
Title: A revision of Dolichogenidea (Hymenoptera, Braconidae, Microgastrinae) with the second mediotergite broadly rectangular from Area de Conservacion Guanacaste, Costa Rica.
Title: Neoboletusantillanus sp. nov. (Boletaceae), first report of a red-pored bolete from the Dominican Republic and insights on the genus Neoboletus.
Title: PITX2 enhances progression of lung adenocarcinoma by transcriptionally regulating WNT3A and activating Wnt/beta-catenin signaling pathway.
Title: miR-328-3p mediates the anti-tumor effect in osteosarcoma via directly targeting MMP-16.
Title: Association of TNFAIP8 gene po

Title: Physical Activity and Sports-Real Health Benefits: A Review with Insight into the Public Health of Sweden.
Title: Potential Prognostic Role of (18)F-FDG PET/CT in Invasive Epithelial Ovarian Cancer Relapse. A Preliminary Study.
Title: Characterization of a Surface Hydrogen Charging Product Affecting the Mechanical Properties in 2205 Duplex Stainless Steel.
Title: Genetic Mutation Analysis of High and Low IgY Chickens by Capture Sequencing.
Title: The Contribution of Surrounding Margins in the Promotion of Natural Enemies in Mediterranean Apple Orchards.
Title: Ceramic Membranes Photocatalytically Functionalized on the Permeate Side and Their Application to Water Treatment.
Title: Which Species Should We Focus On? Umbrella Species Assessment in Southwest China.
Title: Room-Temperature Creep Behavior and Activation Volume of Dislocation Nucleation in a LiTaO3 Single Crystal by Nanoindentation.
Title: Preparation and Characterization of New Environmentally Friendly Starch-Cellulose

Title: The Status of Pro-social Tendency of Left-Behind Adolescents in China: How Family Function and Self-Esteem Affect Pro-social Tendencies.
Title: Why Are People High in Dispositional Awe Happier? The Roles of Meaning in Life and Materialism.
Title: Exploring the Limitations of the Shielding Function of Categorization Rules in Task-Switching.
Title: The Application of Latent Class Analysis for Investigating Population Child Mental Health: A Systematic Review.
Title: Considerations of Mutual Exchange in Prosocial Decision-Making.
Title: Analysis of Moral Disengagement as a Modulating Factor in Adolescents\' Perception of Cyberbullying.
Title: Young-Old City-Dwellers Outperform Village Counterparts in Attention and Verbal Control Tasks.
Title: Students\' Psychological Adjustment in Normative School Transitions From Kindergarten to High School: Investigating the Role of Teacher-Student Relationship Quality.
Title: Ambivalence Predicts Symptomatology in Cognitive-Behavioral and Narrati

In [2]:
#6)Choose a keyword you are interested in (machine learning, computer vision,...). 
#Find the number of articles with the choosen keyword in their meshwords, abstract or title.
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client['homework']
collection = db['pubmed']
keyword = "machine learning"

# Count the number of articles with the keyword in meshwords, abstract, or title
article_count = collection.count_documents({
    "$or": [
        {"meshwords": {"$regex": f".*{keyword}.*", "$options": "i"}},
        {"abstract": {"$regex": f".*{keyword}.*", "$options": "i"}},
        {"title": {"$regex": f".*{keyword}.*", "$options": "i"}}
    ]
})

print(f"Number of articles with the keyword '{keyword}': {article_count}")

Number of articles with the keyword 'machine learning': 521


In [1]:
#7What's the number of articles that have atleast one affiliation AND meshwords.
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client['homework']
collection = db['pubmed']

article_count = collection.count_documents({
    "$and": [
        {"authors": {"$ne": None}},
        {"meshwords": {"$ne": None}}
    ]
})

print(f"Number of articles with at least one affiliation and meshwords: {article_count}")

Number of articles with at least one affiliation and meshwords: 75314


In [1]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client['homework']
collection = db['pubmed']

In [2]:
# 8)Count the number of articles with a publishing date after 2020
article_count = collection.count_documents({
    "date": {"$gt": "year 2020"}
})

print(f"Number of articles with a publishing date after 2020: {article_count}")

Number of articles with a publishing date after 2020: 0


In [21]:
#9)Find articles where there's atleast one affiliation from a choosen country (you decide which one).
# Count the number of articles with at least one affiliation from Canada
article_count = collection.count_documents({
    "authors": {"$regex": "Canada"}})
print("Number of articles with at least one affiliation from Canada:", article_count)
article_country= collection.find({"authors":{"$regex": "Canada"}})
for article in article_country:
    print("ID:", article["_id"],"Title :", article["title"])

Number of articles with at least one affiliation from Canada: 4197
ID: 60c7349bc4b7559e0406e335 Title : Systemic and Metabolic Signature of Sarcopenia in Community-Dwelling Older Adults.
ID: 60c73565c4b7559e04071a8b Title : Cardiorespiratory fitness and cardiovascular disease - The past, present, and future.
ID: 60c7358cc4b7559e04072545 Title : Editorial for the Special Issue on Polymer Based MEMS and Microfabrication.
ID: 60c73595c4b7559e04072723 Title : Correction to: The association between sonographic enthesitis and radiographic damage in psoriatic arthritis.
ID: 60c735d1c4b7559e0407383f Title : Correction to: Rationale and design of the Caloric Restriction and Exercise protection from Anthracycline Toxic Effects (CREATE) study: a 3-arm parallel group phase II randomized controlled trial in early breast cancer.
ID: 60c735d7c4b7559e040739c6 Title : The species of Ecnomocephala Gibson (Hymenoptera: Eupelmidae).
ID: 60c7366ec4b7559e0407632e Title : What\'s in a name? How about being l

ID: 60c73ca4c4b7559e04091413 Title : Clinical medicine and the legacy of the reverend Bayes.
ID: 60c73caac4b7559e0409162e Title : Can Our Guts Tell Us Anything About MS?
ID: 60c73cb3c4b7559e04091876 Title : Patent ductus arteriosus and small for gestational age infants: Treatment approaches and outcomes.
ID: 60c73cbcc4b7559e04091a76 Title : The brown adipose tissue glucagon receptor is functional but not essential for control of energy homeostasis in mice.
ID: 60c73cbcc4b7559e04091aa7 Title : Hepatocellular carcinoma: H-Prune gene regulatory networks.
ID: 60c73cbcc4b7559e04091aa9 Title : General detection of Shiga toxin 2 and subtyping of Shiga toxin 1 and 2 in Escherichia coli using qPCR.
ID: 60c73cc2c4b7559e04091c00 Title : Prostatic ductal adenocarcinoma with cribriform architecture has worse prognostic features than non-cribriform-type.
ID: 60c73ccec4b7559e04091edb Title : Requirement and Duration of Tube Feed Supplementation among Congenital Diaphragmatic Hernia Patients.
ID: 60c7

In [24]:
#10)Check for any duplicates
distinct_pmids = collection.distinct("pmid")
# Check for duplicates
duplicates = len(distinct_pmids) != collection.count_documents({})
# Print the result
if duplicates:
    print("Duplicates found.")
else:
    print("No duplicates found.")

No duplicates found.


In [29]:
#11)Remove every articles where the abstract starts with an "R".
print("init count: ",collection.count_documents({"abstract": {"$regex": "^\\s*\"R"}}))
collection.delete_many({"abstract": {"$regex": "^\\s*\"R"}})
print("after delete many: ",collection.count_documents({"abstract": {"$regex": "^\\s*\"R"}}))

init count:  1918
after delete many:  0


In [ ]:
#12)Return the list of papers (pmid) where there's atleast one affiliation per author
papers = collection.find(
    {"authors": {"$exists": True}}
)
paper_list = [paper["pmid"] for paper in papers if len(paper.get("authors", [])) > 0]
print(paper_list)

In [ ]:
#13)Create 500 random samples of the dataset, 
#compute a statistics that you are interested in and check how it behaves through the different samples
# Set the number of samples
num_samples = 500
statistics = []

for i in range(num_samples):
    sample = collection.aggregate([
        {"$sample": {"size": 100}}  
    ])

    team_sizes = [doc.get("team_size", 0) for doc in sample]
    average_team_size = sum(team_sizes) / len(team_sizes)
    statistics.append(average_team_size)

# Print the statistics( pas print car trop lourd pour github)
print(statistics)

In [44]:
#14) check if there is Univeristy of strasbourg in the affilation
article_unistra = collection.count_documents({
    "authors": {"$regex": "Strasburg"}})
print("Number of articles with at least one affiliation from Unistra:", article_unistra)

Number of articles with at least one affiliation from Unistra: 2


In [45]:
article_country= collection.find({"authors":{"$regex": "Strasburg"}})
for article in article_country:
    print("ID:", article["_id"],"Title :", article["title"])

ID: 60c74063c4b7559e0409fade Title : Circulating aryl hydrocarbon receptor-interacting protein (AIP) is independent of GH secretion.
ID: 60c751b2c4b7559e040f0669 Title : The incidence of adrenal crisis in the postoperative period of HPA axis insufficiency after surgical treatment for Cushing\'s syndrome.
